In [ ]:
using PkgTemplates


In [ ]:
typeof((1,1))

In [ ]:
a::Array{Float64,2} = rand(2,2)

In [ ]:
module tstmod
    include("tmp.jl")
    using .tmpmod
    s = Spectra()
end

In [ ]:
module tstmod2
minmax(x, y) = (y < x) ? (y, x) : (x, y)
gap((min, max)) = max - min
@show gap(minmax(10, 2))
end

In [ ]:
include("tmp.jl")
import .tmpmod
s = Spectra((2,2))

In [ ]:
struct testMat <: AbstractArray{Int,1}
count::Int
# testMat(count::Int) = new(count)
end
Base.size(m::testMat) = (m.count,)
Base.IndexStyle(::Type{<:testMat}) = IndexLinear()
Base.getindex(m::testMat, i::Int) = i*i

In [ ]:
struct testMat2 <: AbstractArray{Int,2}
count::Int
# testMat(count::Int) = new(count)
end
Base.size(m::testMat2) = (m.count, m.count)
Base.IndexStyle(::Type{<:testMat2}) = IndexCartesian()
Base.getindex(m::testMat2, i::Int, j::Int) = i*j
Base.getindex(m::testMat2, i::Float64, j::Float64) = round(i)*round(j)

In [ ]:
struct testMat3 <: AbstractArray{Int,1}
data::Array{Int,1}
end
Base.size(m::testMat3) = size(m.data)
Base.IndexStyle(::Type{<:testMat3}) = IndexLinear()
Base.getindex(m::testMat3, i::Int) = m.data[i]
Base.setindex!(m::testMat3, v, i::Int) = m.data[i] = v


In [ ]:
let a = testMat(5)
    @show a
    a.count = 10
end

In [ ]:
let a = testMat2(5)
    @show a[1.2,2.0]
    a.count = 10
end

In [ ]:
let a = testMat3(rand(Int, 5))
    @show a
    a[2] = 10
    a
end

In [ ]:
struct Params
    logeigvals::StepRangeLen
    logfreevals::StepRangeLen
    constvals::Tuple
end


In [ ]:
const dir = (n=0, u=1, d=2, l=3, r=4)
const 
mutable struct Seeds

end

In [ ]:
struct tst
    a::Int
end
Base.getproperty(t::tst, v::Val{:b}) =  0

In [ ]:
let x = tst(5)
    @show x.a
    @show x.b
end

In [ ]:
:(x, y)

In [ ]:
struct testStruct
    a::Int
    b::Float64
end

In [ ]:
typeof((:x, :y))

In [ ]:
typeof(:(x,y))

In [ ]:
(:(x,y)).args

In [ ]:
:x == :x

In [ ]:
typeof((a=1, b=range(1,2,3)))

In [ ]:
# write a function that takes a named tuple and returns a named tuple with the same names but with the values replaced by the result of a function applied to the values
function mapNamedTuple(f, nt)
    names = fieldnames(typeof(nt))
    vals = getfield.(Ref(nt), names)
    newvals = [typeof(val)<:StepRangeLen ? f.(val) : val for val in vals]
    return NamedTuple{names}(newvals)
end

In [ ]:
mapNamedTuple(x->x.^2, (a=1, b=2, c=range(1,2,3)))

In [ ]:
typeof(range(1,2,3)) <:StepRangeLen

In [6]:
struct testNamedTuple
    nt::NamedTuple
end
# Base.getproperty(t::testNamedTuple, v::Symbol) =  getfield(t.nt, v)
function Base.getproperty(t::testNamedTuple, s::Symbol)
    if s === :nt
        # @show "nt"
        return getfield(t, s)
    else
        # @show "else"
        return getfield(t.nt, s)
    end
end

In [2]:
nt = testNamedTuple((a=1,b=2))

testNamedTuple((a = 1, b = 2))

In [3]:
nt.nt

(a = 1, b = 2)

In [7]:
nt.b

2

In [8]:
dump(nt)

testNamedTuple
  nt: NamedTuple

{(:a, :b), Tuple{Int64, Int64}}
    a: Int64 1
    b: Int64 2


In [ ]:
fieldnames(nt)

In [10]:
"""
    Dirs

    `n`: not defined
    `u`: up(+x2)
    `d`: down(-x2)
    `l`: left(-x1)
    `r`: right(+x1)
"""
@enum Dirs n u d l r
"""
    touchobund(loc, dir)

    return true if the seed is touching the boundary in the direction dir
    return nothing if dir is not defined
"""
function touchobund(loc::Locs, dir::Dirs)
    if dir === n
        return nothing
    elseif dir === u
        return !(0 < loc.x2+1 < loc.size2)
    elseif dir === d
        return !(0 < loc.x2-1 < loc.size2)
    elseif dir === l
        return !(0 < loc.x1-1 < loc.size1)
    elseif dir === r
        return !(0 < loc.x1+1 < loc.size1)
    else
        error("invalid direction")
        return nothing
    end
end
    


In [ ]:
"""
    Locs
    
    `x1`: the position of the seed in the first direction
    `x2`: the position of the seed in the second direction
    `size1`: the number of sites in the first direction
    `size2`: the number of sites in the second direction
    `dir`: the direction of the hopping
"""
mutable struct Locs
    x1::Int
    x2::Int
    #size: the number of sites in each direction
    size1::Int # 0 < x1 < size1
    size2::Int # 0 < x2 < size2
    dir::Dirs
end


In [ ]:
function Base.setproperty!(loc::Locs, s::Symbol, v)
    if s === :x1
        @assert 0 < v < loc.size1 "x1 of loc must be between 0 and size1"
        loc.x1 = Int(v)
    elseif s === :x2
        @assert 0 < v < loc.size2 "x2 of loc must be between 0 and size2"
        loc.x2 = Int(v)
    elseif s === :size1
        @warn "size1 of the Locs struct should not be changed."
        loc.size1 = Int(v)
    elseif s === :size2
        @warn "size2 of the Locs struct should not be changed."
        loc.size2 = Int(v)
    elseif s === :dir
        loc.dir = v
    else
        error("invalid property name")
    end  
end



In [77]:
"""
    extractargswithtype(arglist, type)

    return a symbol vector with the elements of the type
"""
function extractargswithtype(arglist::NamedTuple, type::DataType)
    names = fieldnames(typeof(arglist))
    vals  = getfield.(Ref(arglist), names)
    args = Vector{Symbol}()
    for i in eachindex(vals)
        if typeof(vals[i]) <: type
            push!(args, names[i])
        end
    end
    return args
end

extractargswithtype

In [ ]:

""" 
    Params
    A container that stores the parameters of the model
    construct function
    arglist must have at least two elements
    only two elements in arglist must be of type StepRangeLen
    arglist is a named tuple, so the order of the elements should not be restricted
    the loc argument is construct by default with x1=1, x2=1, size1=lengn(firstStepRangeLenElement), size2=length(secondStepRangeLenElement), dir=r
"""
struct Params
    arglist::NamedTuple
    loc::Locs # the location of the seed
    function Params(arglist::NamedTuple)
        if length(arglist) < 2
            throw(ArgumentError("arglist must have at least two elements"))
        end
        args = extractargswithtype(arglist, StepRangeLen)
        if length(args) != 2
            throw(ArgumentError("only two elements in arglist must be of type StepRangeLen"))
        end
        loc = Locs(1,1,length(arglist[1]),length(arglist[2]),u) # by default with the up direction
        return new(arglist, loc)
    end
    function Params(p::Params)
        return new(p.arglist, deepcopy(p.loc) ) # the loc is independent of the original loc
    end
    
end

In [60]:
# enumerate all possible patterns of the seed
a1 = [true false; false false]
a2 = [true false; true  true ]
a3 = [true true ; false true ]
a4 = [true true ; true  false]
b1 = [true true ; false false]
b2 = [true false; true  false]
b3 = [true false; false true ]

2×2 Matrix{Bool}:
 1  0
 0  1

In [78]:
@assert 0==1

AssertionError: AssertionError: 0 == 1

In [48]:
@enum SpectraElements neg=-1 nan=0 pos=1

In [ ]:
# extract the pattern of seed from data
function extract22pattern(data::Matrix, loc::Locs)
    pattern22 = Matrix{Bool}(undef, 2, 2)
    for x1 in loc.x1 - 1 + 1:2, x2 in loc.x2 - 1 + 1:2
        if data[x1, x2] == nan
            error("data[$x1, $x2] is nan")
        end
        pattern22[x1, x2] = data[x1, x2] == data[loc.x1, loc.x2]
    end
    return pattern22
end

In [59]:
nan*pos

MethodError: MethodError: no method matching *(::SpectraElements, ::SpectraElements)

Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:578


In [50]:
zeros(SpectraElements, 2,2)

MethodError: MethodError: no method matching zero(::Type{SpectraElements})

Closest candidates are:
  zero(!Matched::Union{Type{P}, P}) where P<:Dates.Period
   @ Dates C:\Users\Acer\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\Dates\src\periods.jl:51
  zero(!Matched::LinearAlgebra.UniformScaling{T}) where T
   @ LinearAlgebra C:\Users\Acer\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\LinearAlgebra\src\uniformscaling.jl:135
  zero(!Matched::AbstractIrrational)
   @ Base irrationals.jl:151
  ...


In [75]:
arglist = (a=1, b=range(1,2,3), c=range(1,2,3))
names = fieldnames(typeof(arglist))
vals  = getfield.(Ref(arglist), names)
args = Vector{Symbol}()
for i in eachindex(vals)
    if typeof(vals[i]) <: StepRangeLen
        push!(args, names[i])
    end
end
args
# [typeof(val)<:StepRangeLen for val in vals]
# typeof.(vals) .<:StepRangeLen
# args = names[typeof.(vals) .<:StepRangeLen]

2-element Vector{Symbol}:
 :b
 :c

In [76]:
typeof(Int)

DataType

In [58]:
x = rand(3,3)
for i in (x .< 0.5)
    @show i
end

i = false
i = true
i = true
i = true
i = false
i = false
i = true


i = false
i = true


In [51]:
fill(nan, 2,2)

2×2 Matrix{SpectraElements}:
 nan::SpectraElements = 0  nan::SpectraElements = 0
 nan::SpectraElements = 0  nan::SpectraElements = 0

In [47]:
x = rand(3,3)
x[1+0:2,1]

2-element Vector{Float64}:
 0.5504962499641812
 0.40160628708031487

In [46]:
[i for i in 0:2]

3-element Vector{Int64}:
 0
 1
 2

In [28]:
a1 in [a1,a2,a3,a4]

true

In [13]:
loc = Locs(1,1,3,3, u)

Locs(1, 1, 3, 3, u)

In [19]:
# test that copy works
@show deepcopy(loc).x1 += 1
loc

(deepcopy(loc)).x1 += 1 = 3


Locs(2, 1, 3, 3, u)

In [16]:
loc

Locs(2, 1, 3, 3, u)

In [17]:
Int(u)

1

In [21]:
[1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4